# SVD (Singular Value Decomposition)

$\huge R = P_{mk}S_{kk}Q^T_{kn}$

- https://d3c33hcgiwev3.cloudfront.net/PZRCVq_6EeaoWA59UGLCkA.processed/full/540p/index.mp4?Expires=1513382400&Signature=h0kZOCNli45RxtcBxksVva5SAixX5X0GdmbPCiJMLldT6NInjrFxZoAVoF1JgXBCyMD-B0sjh-Ymo8aMTTDBftY9nWo0fN3d6KO4qdtepcmB3ocof3UQ-58pFvM4PbYLIaYXS0D~bynrjmoEMOukr8~yAc1dKDmXbmNsBrtVRUw_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
- La matriz S es diagonal. Sus valores estan ordenados de mayor a menor. 
- Cada valor de la diagonal de S indica que tan importante es ese feature
- Puedo quedarme con los primeros y aproximar la matriz R
- Tengo que definir las peliculas no calificadas con algun valor 
- Tanto P como Q son ortogonales
- Si R es una matriz real, la decomposición existe
- Elegimos el K y nos garantiza que el la mejor aproximación de rango K (Rank K best aproximation) Trunkated SVD

In [5]:
from scipy.sparse.linalg import svds
import pandas as pd
import numpy as np

In [19]:
def SVD(users_items_train_matrix, k = 20):
    #get SVD components from train matrix. Choose k.
    u, s, vt = svds(users_items_train_matrix, k)
    s_diag_matrix=np.diag(s)
    X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
    return X_pred

In [21]:
def rmse(R, R_estimated):
    # No es conmutativa! Primero va ground truth, normalmente R_test
    # Segundo van las predicciones
    # Es necesario el orden para que el nonzero sea del ground truth
    nonzeros = R.nonzero()
    prediction = R_estimated[nonzeros].flatten()
    ground_truth = R[nonzeros].flatten()
    return np.sqrt(((prediction - ground_truth)**2).sum()/ground_truth.shape[0])

In [22]:
header = ['userId', 'movieId', 'rating', 'timestamp']
df = pd.read_csv('./ml-100k/u.data', sep='\t', names=header)
n_users = df.userId.unique().shape[0]
n_items = df.movieId.unique().shape[0]

In [23]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

In [24]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

# Aplico SVD directamente

In [25]:
SVD_Predictions = SVD(train_data_matrix, k=20)

In [27]:
print('Training:',rmse(train_data_matrix, SVD_Predictions))
print('Testing:',rmse(test_data_matrix, SVD_Predictions))

Training: 2.39053072132499
Testing: 2.712064971324326


# Que pasa si predigo con la media:

In [31]:
mu = train_data_matrix[train_data_matrix.nonzero()].mean()
print(mu)

3.5327466666666667


In [38]:
print('Training:',rmse(train_data_matrix, mu*np.ones(train_data_matrix.shape)))
print('Testing:',rmse(test_data_matrix, mu*np.ones(test_data_matrix.shape)))

Training: 1.1253892611694656
Testing: 1.1265184798799845


# Agrego la media al SVD

In [49]:
train_data_matrix

array([[5., 3., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [53]:
# Resto la media solo a los distinto de cero
R_train_norm = (train_data_matrix - mu)*(train_data_matrix>0) 

In [54]:
SVD_Predictions_norm = SVD(R_train_norm, k=20) + mu

In [55]:
print('Training:',rmse(train_data_matrix, SVD_Predictions_norm))
print('Testing:',rmse(test_data_matrix, SVD_Predictions_norm))

Training: 0.9105254513274095
Testing: 1.050827631113782


# Agrego baselines

In [56]:
from cf_helper_2 import getBaselineEstimates

In [63]:
# Estamos haciendo broadcasting
bii, bui = getBaselineEstimates(train_data_matrix, mu, lambda1 = 0, lambda2 = 0, items_first = True, not_rated = 0)
baseline = mu + bui + bii.T

In [64]:
bui.shape, bii.shape, baseline.shape

((1682, 1), (943, 1), (1682, 943))

In [67]:
R_train_baseline = (train_data_matrix - baseline.T)*(train_data_matrix>0)

In [68]:
SVD_Predictions_baseline = SVD(R_train_baseline, k=20) + baseline.T

In [69]:
print('Training:',rmse(train_data_matrix, SVD_Predictions_baseline))
print('Testing:',rmse(test_data_matrix, SVD_Predictions_baseline))

Training: 0.784368801661053
Testing: 0.9350672849030011
